# Корреляция текущих признаков

In [1]:
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.dummy import DummyClassifier
from sklearn.metrics import f1_score, classification_report, confusion_matrix, precision_score, recall_score
import pandas as pd
import sys

sys.path.append('C:\\Coding\\customer-churn-prediction\\src')
from preprocessing import get_preproc

RND_SEED = 42

In [2]:
telco = pd.read_csv("../data/raw/WA_Fn-UseC_-Telco-Customer-Churn.csv")
y = telco['Churn'].map({'Yes': 1, 'No': 0})
X = telco.drop('Churn', axis=1)

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=RND_SEED, stratify=y
)

In [3]:
import matplotlib.pyplot as plt
import seaborn as sns

preprocessing = get_preproc()
X_train, y_train = preprocessing.fit_resample(X_train, y_train)
telco = X_train.copy()

Columns to drop: ['yes_no__PhoneService', 'yes_no__StreamingMovies', '1hot__gender_Female', '1hot__gender_Male', 'yes_no__MultipleLines', '1hot__PaymentMethod_Mailed check', 'yes_no__OnlineBackup', 'yes_no__DeviceProtection']


In [4]:
telco.head()

,num__MonthlyCharges,num__tenure,num__MonthlyCharges tenure,sqrt__TotalCharges,yes_no__Partner,yes_no__Dependents,yes_no__OnlineSecurity,yes_no__TechSupport,yes_no__StreamingTV,yes_no__PaperlessBilling,1hot__InternetService_DSL,1hot__InternetService_Fiber optic,1hot__InternetService_No,1hot__Contract_Month-to-month,1hot__Contract_One year,1hot__Contract_Two year,1hot__PaymentMethod_Bank transfer (automatic),1hot__PaymentMethod_Credit card (automatic),1hot__PaymentMethod_Electronic check,remainder__SeniorCitizen
0,-0.521976,0.102371,-0.253464,0.002080,0,0,0,0,1,0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0
1,0.337478,-0.711743,-0.514920,-0.295339,1,1,1,0,0,0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0
2,-0.809013,-0.793155,-0.778067,-0.687098,1,1,1,1,0,0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0
3,0.284384,-0.263980,-0.170483,0.099772,1,0,0,0,1,1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0
4,-0.676279,-1.281624,-0.989954,-1.403454,1,1,0,0,0,0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0


In [5]:
telco_corr = pd.concat([telco, y_train], axis=1).corr()

# Строим тепловую карту только для корреляций с целевой переменной
plt.figure(figsize=(20, 20))
telco_corr["Churn"].sort_values(ascending=False)
# sns.heatmap(target_corr, annot=True, cmap='coolwarm', center=0, fmt='.2f')
# plt.title('Корреляция признаков с целевой переменной (Churn)')
# plt.show()

Churn                                            1.000000
1hot__Contract_Month-to-month                    0.483744
1hot__InternetService_Fiber optic                0.350098
1hot__PaymentMethod_Electronic check             0.337950
num__MonthlyCharges                              0.231141
yes_no__PaperlessBilling                         0.219470
remainder__SeniorCitizen                         0.118604
yes_no__StreamingTV                              0.063149
1hot__InternetService_DSL                       -0.152095
1hot__PaymentMethod_Bank transfer (automatic)   -0.153480
1hot__PaymentMethod_Credit card (automatic)     -0.166240
yes_no__Partner                                 -0.194372
1hot__Contract_One year                         -0.218708
num__MonthlyCharges tenure                      -0.221857
yes_no__Dependents                              -0.241903
sqrt__TotalCharges                              -0.243295
yes_no__TechSupport                             -0.248645
yes_no__Online

<Figure size 2000x2000 with 0 Axes>

Отбросим все признаки, не влияющие особо на таргет.

In [ ]:
for col in target_corr.index:
    if abs(target_corr[col]) < 0.1:
        telco.drop(columns=col, inplace=True)
telco.info()